In [1]:
import tensorflow as tf
import numpy as np

In [2]:
def add_layer(inputs,in_size,out_size,n_layer,activation_function = None):
    layer_name = 'layer%s'% n_layer
    with tf.name_scope('layer'):
        with tf.name_scope('weight'):
            Weights = tf.Variable(tf.random_normal([in_size,out_size]),name='W')
            tf.summary.histogram(layer_name+'/Weights',Weights)
        with tf.name_scope('biases'):
            biases = tf.Variable(tf.zeros([1,out_size])+0.1,name='b')
            tf.summary.histogram(layer_name+'/biases',biases)
        
        with tf.name_scope('Wx_plus_b'):
             Wx_plus_b = tf.matmul(inputs,Weights)+ biases       
        if activation_function is None:
            outputs = Wx_plus_b
        else:
            outputs = activation_function(Wx_plus_b)
        tf.summary.histogram(layer_name+'/outputs',outputs)
    return outputs

## Make up some real data

In [3]:
x_data = np.linspace(-1,1,300)[:,np.newaxis]
noise = np.random.normal(0,0.05,x_data.shape)
y_data = np.square(x_data)-0.5 +noise
print(x_data.shape,y_data.shape)

(300, 1) (300, 1)


## Modeling

* define placeholder

In [4]:
with tf.name_scope('inputs'):
    xs = tf.placeholder(tf.float32,[None,1],name = 'x_input')
    ys = tf.placeholder(tf.float32,[None,1],name = 'y_input')
#None indicates that the first dimension, corresponding to the batch size, can be of any size.

* add hidden layer

In [5]:
l1 = add_layer(xs,1,10,n_layer=1,activation_function =tf.nn.relu)

* add output layer

In [6]:
prediction = add_layer(l1,10,1,n_layer=2,activation_function = None)

* the error between prediction and real data
> indices mean dimensions chosed to be calculated

In [7]:
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.reduce_sum(tf.square(ys-prediction),
                                    reduction_indices = [1]))
    tf.summary.scalar('loss',loss)

with tf.name_scope('train'):
    train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

* important step

In [8]:
init = tf.global_variables_initializer()

## Training
* merge_all for collecting output data

In [9]:
sess = tf.Session()
merged = tf.summary.merge_all()

writer = tf.summary.FileWriter('log/',sess.graph)

In [10]:
sess.run(init)

In [11]:
for i in range(1000):
    sess.run(train_step,feed_dict={xs:x_data,ys:y_data})
    if i % 10 ==0:
        result = sess.run(merged,feed_dict={xs:x_data,ys:y_data})
        writer.add_summary(result,i)

## Visualize

cd C:\Users\Aramis\Documents\NB_TensorFlow\log

tensorboard --logdir=log

http://219.216.80.138:6006